## Predviđanje strukture *RNK*<a id="par:toc"></a>

U radu je razmotrena primena [probabilističkih kontekstno slobodnih gramatika](https://en.wikipedia.org/wiki/Probabilistic_context-free_grammar) (*PCFG*) i [konvolutivnih neuronskih mreža](https://en.wikipedia.org/wiki/Convolutional_neural_network) (*CNN*) u predviđanju sekundarne strukture [ribonukleinske kiseline](https://en.wikipedia.org/wiki/RNA) (*RNK*). Preuzet je skup *RNK* sa pridruženim strukturama, nakon čega je na delu podataka obučena po jedna stohastička gramatika i duboka konvolutivna mreža. Modeli su vrednovani prema uspehu na drugom delu podataka, odnosno sličnosti predviđanja sa stvarnošću.

## Sadržaj

1. [Struktura *RNK*](#par:strukt)

## 1. Struktura *RNK* [⮭]<a id="par:strukt"></a>

[⮭]: #par:toc

*RNK* je biopolimer čiju [primarnu strukturu](https://en.wikipedia.org/wiki/Nucleic_acid_sequence) gradi niz [nukleobaza](https://en.wikipedia.org/wiki/Nucleobase). U sastav sekvence ulaze četiri kanonske nukleobaze – [adenin](https://en.wikipedia.org/wiki/Adenine), [uracil](https://en.wikipedia.org/wiki/Uracil), [guanin](https://en.wikipedia.org/wiki/Guanine) i [citozin](https://en.wikipedia.org/wiki/Cytosine). Računarski gledano, *RNK* se stoga najčešće predstavlja kao niska nad azbukom $\{A, U, C, G\}$, što je skup početnih slova navedenih baza.

<img src='../slike/primarna.png' width='40%'>

Specijalno, sekvencu mogu činiti i drugi karakteri, koji predstavljaju [modifikovane baze](https://en.wikipedia.org/wiki/Nucleobase#Modified_nucleobases). Tako, na primer, [pseudouridin](https://en.wikipedia.org/wiki/Pseudouridine) (*P*) predstavlja izmenjeni [uridin](https://en.wikipedia.org/wiki/Uridine) (*U*). Ovakva modifikovanost ne doprinosi problemu [predviđanja strukture](https://en.wikipedia.org/wiki/Nucleic_acid_structure_prediction) i najčešće se zanemaruje, te nije ni predstavljena u poznatim bazama podataka.

Kako bi prešla u biološki aktivnu formu, *RNK* se [uvija intramolekularno](https://en.wikipedia.org/wiki/Nucleic_acid_structure), čime formira segmente uparenih nukleobaza. Takvi segmenti nazivaju se zavojnicama (*helix*) ili drškama (*stem*), dok nizovi neuparenih baza grade petlje (*loop*). U zavisnosti od oblika, postoji više različitih vrsta petlji: ukosnica (šnala, *hairpin*), unutrašnja (*interior*), razgranata (*multi-branched*), ispupčenje (*bulge*). Posebno se izdvaja i spoljašnji nestrukturirani lanac (*unstructured single strand*).

<img src='../slike/sekundarna.png' width='70%'>

Uvijanje se najčešće vizuelizuje u formi [grafa](https://en.wikipedia.org/wiki/Graph_%28discrete_mathematics%29), pri čemu su čvorovi baze, dok grane postoje između uparenih baza (označene kružićima ili lukovima), kao i između susednih elemenata sekvence (implicitne, neoznačene). Pored realističnog prikaza (*c*), popularni su i oni koji sekvencu predstavljaju na jednoj pravoj (*a*) ili na kružnici (*b*). Oni su značajni jer svedoče o ugnežđenosti uparivanja. Važan je i prikaz u formi matrice povezanosti (*d*), o čemu će biti reči u nastavku.

<img src='../slike/vizuelizacija.png' width='90%'>

Ovako formirani [bazni parovi](https://en.wikipedia.org/wiki/Base_pair) upravo čine [sekundarnu strukturu](https://en.wikipedia.org/wiki/Nucleic_acid_secondary_structure) *RNK*. Parovi *A−U* i *G−C* pritom su kanonski (prema Votsonu i Kriku), mada su moguće i druge ([nekanonske](https://en.wikipedia.org/wiki/Non-canonical_base_pairing)) varijante uparivanja, pri čemu se kao treći posebno izdvaja nešto manje stabilan ([*wobble*](https://en.wikipedia.org/wiki/Wobble_base_pair)), ali i dalje termodinamički povoljan par *U−G*.

<img src='../slike/familija.png' width='60%'>

Osim u formi grafa ili prostog spiska parova, sekundarnu strukturu moguće je predstaviti i kao nisku, i to nad azbukom koja se sastoji od [tačke i zagrada](https://eprints.soton.ac.uk/267292/1/RamlanEI08ExtDotBracRNA.pdf). U ovakvoj notaciji, tačka predstavlja neuparenu bazu, dok zagrade označavaju parove. Uparivanje baza, očekivano, prati matematičko i jezičko pravilo uparivanja zagrada. Podrazumevano, niske koje predstavljaju primarnu (sekvencu) i sekundarnu (uparivanje) strukturu iste *RNK* moraju biti saglasne.

<img src='../slike/pseudocvor.png' width='40%'>

U sastav sekundarne strukture ulazi nekoliko značajnih motiva. Posebno je zanimljiv strukturni motiv [ukosnice sa drškom](https://en.wikipedia.org/wiki/Stem-loop) (*stem-loop*), koji karakteriše sve ukosnice. Njegova važna odlika je da petlja sadrži najmanje tri baze, što je ograničenje koje je neophodno ugraditi u računarske metode. [Pseudočvorovi](https://en.wikipedia.org/wiki/Pseudoknot) su još jedna znamenita pojava, budući da predstavljaju neugnežđena uparivanja. Oni, međutim, zadiru u [tercijarnu](https://en.wikipedia.org/wiki/Nucleic_acid_tertiary_structure) i [kvaternarnu](https://en.wikipedia.org/wiki/Nucleic_acid_quaternary_structure) strukturu, koje nisu razmatrane.